In [8]:
from client import Client
from global_model import Global_model
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [9]:
selected_model = Supported_modles.gradient_boosting_classifier

In [10]:
#NODEs
client1 = Client("node1","0.0.0.0")
dataset = client1.load_data("../../datasets/UNSW_NB15_training-set.csv")
client2 = Client("node2","0.0.0.0")
client3 = Client("node3","0.0.0.0")
client4 = Client("node4","0.0.0.0")
client5 = Client("node5","0.0.0.0")

dataset = shuffle(dataset)

clients = [client1, client2, client3, client4, client5]

client1.preprocess_data(dataset)

X = client1.x
y = client1.y

client1.x = X[:20000]
client1.y = y[:20000]

client2.x = X[20000:40000]
client2.y = y[20000:40000]

client3.x = X[40000:50000]
client3.y = y[40000:50000]

client4.x = X[50000:60000]
client4.y = y[50000:60000]

client5.x = X[60000:]
client5.y = y[60000:]

In [11]:
f1_before = []
for client in clients:
    client.split_data()
    client.train_model(selected_model)
    print(f'Model accuracy: {client.accuracy}')
    print(f'Model f1 score: {client.F1}')
    f1_before.append(client.F1)
    print(f'Total size: {client.x.shape[0]}')
    print(f'Totla sum of malicious packets: {client.y.sum()}')

Model accuracy: 0.9345454545454546
Model f1 score: 0.941031941031941
Total size: 13400
Totla sum of malicious packets: 7394
Model accuracy: 0.9513636363636364
Model f1 score: 0.9551111732624807
Total size: 13400
Totla sum of malicious packets: 7350
Model accuracy: 0.9484848484848485
Model f1 score: 0.953219592735278
Total size: 6700
Totla sum of malicious packets: 3658
Model accuracy: 0.9466666666666667
Model f1 score: 0.9505895564289726
Total size: 6700
Totla sum of malicious packets: 3682
Model accuracy: 0.9489823609226594
Model f1 score: 0.9539328595932369
Total size: 14962
Totla sum of malicious packets: 8267


In [5]:
client1.model.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 1.0,
 'loss': 'deviance',
 'max_depth': 1,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 0,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [6]:
def bag_models(clients,data):
    result = []
    for x in data:
        c1 = 0
        c0 = 0
        for client in clients:
            c1 = c1 + (client.model.predict_proba([x])[0][1])
            c0 = c0 + (client.model.predict_proba([x])[0][0])
        if c1 > c0:
            result.append(1)
        else:
            result.append(0)
    return result

In [7]:
for client in clients:
    result = bag_models(clients,client.x_test)
    y_hat = client.model.predict(client.x_test)
    print(f1_score(client.y_test,result) - f1_score(client.y_test,y_hat))
    print("+++++++++++++++++++++")

0.02204819810631109
+++++++++++++++++++++
-0.0006512547735952889
+++++++++++++++++++++
0.003378519670148794
+++++++++++++++++++++
-0.005208790088206849
+++++++++++++++++++++
-0.0021226593866008336
+++++++++++++++++++++
